<font size="8"><b>PREDICTING FLIGHT DELAYS</b></font>

<font size="6"><b>AdaBoost - with DEP_DELAY</b></font>

![Figure_6](img/Figure_6.png)

In [1]:
import pandas as pd
import numpy as np
np.random.seed(0)
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import xgboost as xgb
sns.set_style('darkgrid')
pd.set_option('display.max_columns', None)
import datetime, warnings, scipy
warnings.filterwarnings("ignore")

from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn import svm
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report

In [2]:
dfm_ready = pd.read_csv('dfm_ready.csv', index_col=0)
dfm_ready.head()

,DEP_DELAY,CRS_ELAPSED_TIME,AIR_TIME,DISTANCE,FLIGHT_STATUS,OP_CARRIER_Allegiant Air,OP_CARRIER_American Airlines,OP_CARRIER_Delta Airlines,OP_CARRIER_Endeavor Air,OP_CARRIER_Envoy Air,OP_CARRIER_ExpressJet,OP_CARRIER_Frontier Airlines,OP_CARRIER_Hawaiian Airlines,OP_CARRIER_JetBlue Airways,OP_CARRIER_Mesa Airline,OP_CARRIER_PSA Airlines,OP_CARRIER_Republic Airways,OP_CARRIER_SkyWest Airlines,OP_CARRIER_Southwest Airlines,OP_CARRIER_Spirit Airlines,OP_CARRIER_United Airlines,OP_CARRIER_Virgin America,DEST_Atlanta,DEST_Boston,DEST_Charlotte,DEST_Chicago,DEST_Dallas-Fort Worth,DEST_Denver,DEST_Detroit,DEST_Houston,DEST_Las Vegas,DEST_Los Angeles,DEST_Minneapolis,DEST_New York,DEST_Newark,DEST_Orlando,DEST_Philadelphia,DEST_Phoenix,DEST_Salt Lake City,DEST_San Francisco,DEST_Seattle,CRS_DEP_TIME_2,CRS_DEP_TIME_3,CRS_DEP_TIME_4,CRS_ARR_TIME_2,CRS_ARR_TIME_3,CRS_ARR_TIME_4,MONTH_2,MONTH_3,MONTH_4,MONTH_5,MONTH_6,MONTH_7,MONTH_8,MONTH_9,MONTH_10,MONTH_11,MONTH_12,WEEKDAY_1,WEEKDAY_2,WEEKDAY_3,WEEKDAY_4,WEEKDAY_5,WEEKDAY_6
0,-5.0,268.0,225.0,1605.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,-8.0,99.0,65.0,414.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,-5.0,134.0,106.0,846.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,6.0,190.0,157.0,1120.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,-3.0,206.0,173.0,1222.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [3]:
dfm_ready.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4008257 entries, 0 to 7213445
Data columns (total 64 columns):
 #   Column                         Dtype  
---  ------                         -----  
 0   DEP_DELAY                      float64
 1   CRS_ELAPSED_TIME               float64
 2   AIR_TIME                       float64
 3   DISTANCE                       float64
 4   FLIGHT_STATUS                  int64  
 5   OP_CARRIER_Allegiant Air       int64  
 6   OP_CARRIER_American Airlines   int64  
 7   OP_CARRIER_Delta Airlines      int64  
 8   OP_CARRIER_Endeavor Air        int64  
 9   OP_CARRIER_Envoy Air           int64  
 10  OP_CARRIER_ExpressJet          int64  
 11  OP_CARRIER_Frontier Airlines   int64  
 12  OP_CARRIER_Hawaiian Airlines   int64  
 13  OP_CARRIER_JetBlue Airways     int64  
 14  OP_CARRIER_Mesa Airline        int64  
 15  OP_CARRIER_PSA Airlines        int64  
 16  OP_CARRIER_Republic Airways    int64  
 17  OP_CARRIER_SkyWest Airlines    int64  
 18  OP

In this second version of thee XGBoost, I will drop the DEP_DELAY just as a test. However, I do believe that the model should have it as this is a feature that you normally know before you board the plane and it does not necessarily suggest a delay at the arrival destination. 

There are a few factors that wee should have considered while cleaning the data and could have affected these decisions. These factors are that according to the OAG, a flight is not considered delayed from the departure nor the arrival city, if it is lower or equal than 15 minutes. That could have affected the EDA that I did plus the decisions made for the models in terms of variables. So it will definitely be a suggestion for a way forward.

You can look at the source of the information <a href="https://www.oag.com/airline-on-time-performance-defining-late">here</a>

# AdaBoost

In [4]:
# Instantiate an AdaBoosClassifier
adaboost_clf = AdaBoostClassifier(random_state=42)
                                 
# Instantiate an GradientBoostingClassifier
gbt_clf = GradientBoostingClassifier(random_state=42)

In [5]:
# Define features (X) and traget(y)
y = dfm_ready['FLIGHT_STATUS']
X = dfm_ready.drop(['FLIGHT_STATUS'], axis = 1)

In [7]:
# Perform the dataset split
X_test, X_train, y_test, y_train = train_test_split(X, y, test_size=0.25, random_state=42)

In [8]:
adaboost_clf.fit(X_train, y_train)

AdaBoostClassifier(random_state=42)

In [9]:
gbt_clf.fit(X_train, y_train)

GradientBoostingClassifier(random_state=42)

In [10]:
# Using the models to predict labels on both training and test sets

# AdaBoost model predictions
adaboost_train_preds = adaboost_clf.predict(X_train)
adaboost_test_preds = adaboost_clf.predict(X_test)

# GradientBoosting model predictions
gbt_clf_train_preds = gbt_clf.predict(X_train)
gbt_clf_test_preds = gbt_clf.predict(X_test)

In [19]:
# Calculating the accuracy and f1-score for each model:

def display_acc_and_f1_score(true, preds, model_name):
    acc = accuracy_score(true, preds)
    f1 = f1_score(true, preds)
    print("Model: {}".format(model_name))
    print("Accuracy: {}".format(acc))
    print("F1-Score: {}".format(f1))
    
print("Training Metrics")
print('----------------------------')
display_acc_and_f1_score(y_train, adaboost_train_preds, model_name='AdaBoost')
print("")
display_acc_and_f1_score(y_train, gbt_clf_train_preds, model_name='Gradient Boosted Trees')
print("")
print("Testing Metrics")
print('----------------------------')
display_acc_and_f1_score(y_test, adaboost_test_preds, model_name='AdaBoost')
print("")
display_acc_and_f1_score(y_test, gbt_clf_test_preds, model_name='Gradient Boosted Trees')

Training Metrics
----------------------------
Model: AdaBoost
Accuracy: 0.8172882996611996
F1-Score: 0.7022851112473393

Model: Gradient Boosted Trees
Accuracy: 0.8318322663699461
F1-Score: 0.7305988486324113

Testing Metrics
----------------------------
Model: AdaBoost
Accuracy: 0.816682034946537
F1-Score: 0.7009717662009417

Model: Gradient Boosted Trees
Accuracy: 0.8309053447018687
F1-Score: 0.7286903046990878


Confusion Matrix 

In [12]:
adaboost_confusion_matrix = confusion_matrix(y_test, adaboost_test_preds)
adaboost_confusion_matrix

array([[1809181,   80227],
       [ 470862,  645922]])

In [13]:
gbt_confusion_matrix = confusion_matrix(y_test, gbt_clf_test_preds)
gbt_confusion_matrix

array([[1815217,   74191],
       [ 434140,  682644]])

In [14]:
adaboost_classification_report = classification_report(y_test, adaboost_test_preds)
print(adaboost_classification_report)

              precision    recall  f1-score   support

           0       0.79      0.96      0.87   1889408
           1       0.89      0.58      0.70   1116784

    accuracy                           0.82   3006192
   macro avg       0.84      0.77      0.78   3006192
weighted avg       0.83      0.82      0.81   3006192



In [15]:
gbt_classification_report = classification_report(y_test, gbt_clf_test_preds)
print(gbt_classification_report)

              precision    recall  f1-score   support

           0       0.81      0.96      0.88   1889408
           1       0.90      0.61      0.73   1116784

    accuracy                           0.83   3006192
   macro avg       0.85      0.79      0.80   3006192
weighted avg       0.84      0.83      0.82   3006192



In [16]:
print('Mean Adaboost Cross-Val Score (k=5):')
print(cross_val_score(adaboost_clf, X, y, cv=5).mean())

Mean Adaboost Cross-Val Score (k=5):
0.7603938514565318


In [17]:
print('Mean GBT Cross-Val Score (k=5):')
print(cross_val_score(gbt_clf, X, y, cv=5).mean())

Mean GBT Cross-Val Score (k=5):
0.7727191597199039
